# Praca domowa 4
### Krzysztof Tabeau


Wstęp:

W poniższym raporcie są zamieszczone wyniki 3 zapytań
rozwiązanych na dwa sposoby. Pierwszym z nich jest skorzystanie z funkcji  pandas.read_sql_query(),
która interpretuje zapytanie napisane w SQL, a drugą użycie funkcji bazowych, funkcji z biblioteki numpy oraz pandas do przetwarzania danych.
Następnie będzie pokazany typ wyników za pomocą funkcji oraz sprawdzenie równoważności rozwiązania funkcją equals().

#### 0. Import bibliotek oraz danych i stworzenie bazy danych

In [325]:
import pandas as pd
import numpy as np
import os, os.path
import tempfile, sqlite3
import copy
tempfile.mkdtemp()

pd.set_option("display.notebook_repr_html", True)
Badges = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Badges.csv.gz",compression = "gzip")
Comments = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Comments.csv.gz",compression = "gzip")
PostLinks = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/PostLinks.csv.gz",compression = "gzip")
Posts = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Posts.csv.gz",compression = "gzip")
Tags = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Tags.csv.gz",compression = "gzip")
Users = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Users.csv.gz",compression = "gzip")
Votes = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Votes.csv.gz",compression = "gzip")

baza = os.path.join(tempfile.mkdtemp(), "travel.db")
if os.path.isfile(baza):
    os.remove(baza)

conn = sqlite3.connect(baza)

Badges.to_sql("Badges", conn, if_exists="replace")
Comments.to_sql("Comments", conn, if_exists="replace")
PostLinks.to_sql("PostLinks", conn, if_exists="replace")
Posts.to_sql("Posts", conn, if_exists="replace")
Tags.to_sql("Tags", conn, if_exists="replace")
Users.to_sql("Users", conn, if_exists="replace")
Votes.to_sql("Votes", conn, if_exists="replace")

#### 1. Zapytanie zwraca pytanie i jego największą liczbę łapek w górę jaką dostał w jednym roku oraz informacje w którym to było roku

##### 1.1 Zapytanie za pomocą pandas.read_sql_query

In [326]:
result1 = pd.read_sql_query("""
                            SELECT
                                Posts.Title,
                                UpVotesPerYear.Year,
                                MAX(UpVotesPerYear.Count) AS Count
                            FROM (
                                SELECT
                                    PostId,
                                    COUNT(*) AS Count,
                                    STRFTIME('%Y', Votes.CreationDate) AS Year
                                FROM Votes
                                WHERE VoteTypeId=2
                                GROUP BY PostId, Year
                            ) AS UpVotesPerYear
                            JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
                            WHERE Posts.PostTypeId=1
                            GROUP BY Year
                    """, conn)

print(result1)

                                               Title  Year  Count
0  OK we're all adults here, so really, how on ea...  2011     70
1    How to successfully haggle / bargain in markets  2012     37
2  Why are airline passengers asked to lift up wi...  2013    103
3  How do you know if Americans genuinely/literal...  2014    180
4  Immigration officer that stopped me at the air...  2015    119
5  I don't know my nationality. How can I visit D...  2016    135
6                       Why prohibit engine braking?  2017    178


##### 1.2 Zapytanie za pomocą zwykłych funkcji

In [327]:
UpVotesPerYear = copy.deepcopy(Votes)
UpVotesPerYear["CreationDate"] = pd.to_datetime(UpVotesPerYear.CreationDate).dt.strftime("%Y")
UpVotesPerYear = UpVotesPerYear\
    .rename(columns = {"CreationDate":"Year"})\
    .loc[UpVotesPerYear.VoteTypeId == 2, ["PostId","Year"]]\
    .groupby(["PostId","Year"])\
    .size()\
    .reset_index()\
    .rename(columns = {0:"Count"})

Posts2 = copy.deepcopy(Posts)
Posts2 = Posts2\
    .rename(columns = {"Id":"PostId"})\
    .loc[Posts2.PostTypeId == 1]
Result = UpVotesPerYear\
    .merge(Posts2, on="PostId")[["Title","Year","Count"]]
Result = Result\
    .groupby(["Year"])["Count"]\
    .agg(np.max)\
    .reset_index()\
    .merge(Result, on=["Year","Count"])
result2 = Result[["Title","Year","Count"]]
print(result2)

                                               Title  Year  Count
0  OK we're all adults here, so really, how on ea...  2011     70
1    How to successfully haggle / bargain in markets  2012     37
2  Why are airline passengers asked to lift up wi...  2013    103
3  How do you know if Americans genuinely/literal...  2014    180
4  Immigration officer that stopped me at the air...  2015    119
5  I don't know my nationality. How can I visit D...  2016    135
6                       Why prohibit engine braking?  2017    178


##### 1.3 Sprawdzenie typów wyników

In [328]:
print(result1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   7 non-null      object
 1   Year    7 non-null      object
 2   Count   7 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 296.0+ bytes
None


In [329]:
print(result2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   7 non-null      object
 1   Year    7 non-null      object
 2   Count   7 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 224.0+ bytes
None


##### 1.4 Sprawdzenie równoważności wyników

In [330]:
print(result1.equals(result2))

True


#### 2. Zapytanie zwraca id, tytuł i liczbę pozytywnych odpowiedzi pytań z największą liczbą pozytywnych odpowiedzi

##### 2.1 Zapytanie za pomocą pandas.read_sql_query

In [331]:
result1 = pd.read_sql_query("""
                            SELECT
                                Posts.ID,
                                Posts.Title,
                                Posts2.PositiveAnswerCount
                            FROM Posts
                            JOIN (
                                SELECT
                                Posts.ParentID,
                                COUNT(*) AS PositiveAnswerCount
                                FROM Posts
                                WHERE Posts.PostTypeID=2 AND Posts.Score>0
                                GROUP BY Posts.ParentID
                            ) AS Posts2
                            ON Posts.ID=Posts2.ParentID
                            ORDER BY Posts2.PositiveAnswerCount DESC
                            LIMIT 10
                    """, conn)

print(result1)

      Id                                              Title  \
0    250  Which European cities have bike rental station...   
1     10  When traveling to a country with a different c...   
2  13562    How do you choose a restaurant when travelling?   
3  48775  How can I deal with people asking to switch se...   
4   3220        Why would you wrap your luggage in plastic?   
5  43660  Traveling in Europe Solo - 18 years old. Feasi...   
6  30656                      Long-life SIM cards in Europe   
7   7663         Am I expected to tip wait staff in Europe?   
8  59128  Is there a way to prevent "looking like a tour...   
9  60446  Is it rude to ask if the food contains pork or...   

   PositiveAnswerCount  
0                   24  
1                   20  
2                   20  
3                   20  
4                   19  
5                   19  
6                   18  
7                   17  
8                   17  
9                   17  


##### 2.2 Zapytanie za pomocą zwykłych funkcji

In [332]:
Posts2 = copy.deepcopy(Posts)
Posts2 = Posts2\
    .loc[(Posts2.PostTypeId == 2) & (Posts2.Score > 0)]\
    .groupby(["ParentId"])\
    .size()\
    .reset_index()\
    .rename(columns = {0:"PositiveAnswerCount", "ParentId":"Id"})\
    .merge(Posts, on="Id")[["Id","Title","PositiveAnswerCount"]]\
    .sort_values(["PositiveAnswerCount","Id"], ascending=[False,True])\
    .head(10)\
    .reset_index(drop = True)
Posts2["Id"] = Posts2["Id"].astype(np.int64)
result2 = Posts2
print(result2)

      Id                                              Title  \
0    250  Which European cities have bike rental station...   
1     10  When traveling to a country with a different c...   
2  13562    How do you choose a restaurant when travelling?   
3  48775  How can I deal with people asking to switch se...   
4   3220        Why would you wrap your luggage in plastic?   
5  43660  Traveling in Europe Solo - 18 years old. Feasi...   
6  30656                      Long-life SIM cards in Europe   
7   7663         Am I expected to tip wait staff in Europe?   
8  59128  Is there a way to prevent "looking like a tour...   
9  60446  Is it rude to ask if the food contains pork or...   

   PositiveAnswerCount  
0                   24  
1                   20  
2                   20  
3                   20  
4                   19  
5                   19  
6                   18  
7                   17  
8                   17  
9                   17  


##### 2.3 Sprawdzenie typów wyników

In [333]:
print(result1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   10 non-null     int64 
 1   Title                10 non-null     object
 2   PositiveAnswerCount  10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes
None


In [334]:
print(result2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   10 non-null     int64 
 1   Title                10 non-null     object
 2   PositiveAnswerCount  10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes
None


##### 2.4 Sprawdzenie równoważności wyników

In [335]:
print(result1.equals(result2))

True


#### 3. Zapytanie zwraca informację, które są dostępne, o użytkownikach (id, nick, reputacja, wiek, lokacja), którzy mają 2 - 10 odznak pierwszej klasy.

##### 3.1 Zapytanie za pomocą pandas.read_sql_query

In [336]:
result1 = pd.read_sql_query("""
                            SELECT DISTINCT
                                Users.Id,
                                Users.DisplayName,
                                Users.Reputation,
                                Users.Age,
                                Users.Location
                            FROM (
                                SELECT
                                    Name, UserID
                                    FROM Badges
                                WHERE Name IN (
                                    SELECT
                                        Name
                                    FROM Badges
                                    WHERE Class=1
                                    GROUP BY Name
                                    HAVING COUNT(*) BETWEEN 2 AND 10
                                )
                                AND Class=1
                            ) AS ValuableBadges
                            JOIN Users ON ValuableBadges.UserId=Users.Id
                    """, conn)

print(result1)

      Id         DisplayName  Reputation   Age  \
0    108      Ankur Banerjee       31273  27.0   
1     19               VMAtm       18556  33.0   
2    101           Mark Mayo      121667  37.0   
3    466      iHaveacomputer        8360   NaN   
4    793       mindcorrosive       10531  32.0   
5    693  RoflcoptrException       33300   NaN   
6   6669             Relaxed       69405   NaN   
7   1737           Gayot Fow       70237   NaN   
8  39065                Pont        1004   NaN   
9  19400               phoog       34342  50.0   

                             Location  
0                          London, UK  
1            Tampa, FL, United States  
2  Sydney, New South Wales, Australia  
3                        Down underer  
4                            Bulgaria  
5                                None  
6                                None  
7              London, United Kingdom  
8                             Austria  
9                        New York, NY  


##### 3.2 Zapytanie za pomocą zwykłych funkcji

In [337]:
Names = copy.deepcopy(Badges)
Names = Names\
    .loc[Names.Class == 1]\
    .groupby(["Name"])\
    .size()\
    .reset_index()\
    .rename(columns = {0:"Count"})
Names = Names\
    .loc[Names.Count.between(2,10)]\
    .reset_index(drop = True)

Names = Names.Name.to_list()

ValuableBadges = copy.deepcopy(Badges)
result2 = ValuableBadges.loc[(ValuableBadges.Class == 1) & (ValuableBadges.Name.isin(Names)),["Name","UserId"]]\
    .reset_index(drop = True)\
    .rename(columns = {"UserId":"Id"})\
    .merge(Users, on = "Id")[["Id","DisplayName","Reputation","Age","Location"]]\
    .drop_duplicates()\
    .reset_index(drop = True)

print(result2)

      Id         DisplayName  Reputation   Age  \
0    108      Ankur Banerjee       31273  27.0   
1     19               VMAtm       18556  33.0   
2    101           Mark Mayo      121667  37.0   
3    466      iHaveacomputer        8360   NaN   
4    793       mindcorrosive       10531  32.0   
5    693  RoflcoptrException       33300   NaN   
6   6669             Relaxed       69405   NaN   
7   1737           Gayot Fow       70237   NaN   
8  39065                Pont        1004   NaN   
9  19400               phoog       34342  50.0   

                             Location  
0                          London, UK  
1            Tampa, FL, United States  
2  Sydney, New South Wales, Australia  
3                        Down underer  
4                            Bulgaria  
5                                 NaN  
6                                 NaN  
7              London, United Kingdom  
8                             Austria  
9                        New York, NY  


##### 3.3 Sprawdzenie typów wyników

In [338]:
print(result1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           10 non-null     int64  
 1   DisplayName  10 non-null     object 
 2   Reputation   10 non-null     int64  
 3   Age          5 non-null      float64
 4   Location     8 non-null      object 
dtypes: float64(1), int64(2), object(2)
memory usage: 528.0+ bytes
None


In [339]:
print(result2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           10 non-null     int64  
 1   DisplayName  10 non-null     object 
 2   Reputation   10 non-null     int64  
 3   Age          5 non-null      float64
 4   Location     8 non-null      object 
dtypes: float64(1), int64(2), object(2)
memory usage: 528.0+ bytes
None


##### 3.4 Sprawdzenie równoważności wyników

In [340]:
print(result1.equals(result2))

True


## Koniec